# Lesson 4a: Data cleaning and Preprocessing

Real world datasets often contain issues like missing values, duplicate rows and inconsistent formatting. Before preforming analysis or modeling it's important to clean and preprocess the data to ensure accuracy and reliability.

Data cleaning is the process of dropping any null values, inconsistent or dirty data.
We can spend a lot of time cleaning datasets, so its really important to know how to do it properly. We will cover the following:
- Dropping unnecessary columns in a DataFrame
- Changing the index of a DataFrame
- Using .str() methods to clean columns
- Using the DataFrame.applymap() function to clean the entire dataset, element-wise
- Renaming columns to a more recognizable set of labels
- Skipping unnecessary rows in a CSV file



### 📥 Getting the Tutorial Data

Before we begin, we need to download the example data files used in this tutorial.  
The following function will:
- Clone a GitHub repository containing the data,
- Remove any old copies to avoid conflicts,
- Copy `.txt` and `.csv` files into the current working directory.


We will use 3 datasets for this example:
- `BL-Flickr-Images-Book.csv` – A CSV file containing information about books from the British Library
- `university_towns.txt` – A text file containing names of college towns in every US state
- `olympics.csv` – A CSV file summarizing the participation of all countries in the Summer and Winter Olympics


Run the cell below to set up your environment.

In [6]:
def fetch_data():
  import os, shutil
  cwd = os.getcwd()
  if os.path.exists("CosmicAI_WinterSchool"):
    shutil.rmtree("CosmicAI_WinterSchool")
  !git clone https://github.com/aliawofford9317/CosmicAI_WinterSchool.git
  for file in os.listdir("CosmicAI_WinterSchool"):
    if file.endswith((".txt",".csv")):
      shutil.copy("CosmicAI_WinterSchool/{}".format(file),cwd)
fetch_data()

Cloning into 'CosmicAI_WinterSchool'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 79 (delta 21), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (79/79), 8.50 MiB | 9.56 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [7]:
# Let's import the required modules
import pandas as pd
import numpy as np

First Let's open our data:

In [8]:
df = pd.read_csv('BL-Flickr-Images-Book.csv')
df.head()

,Identifier,Edition Statement,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Corporate Author,Corporate Contributors,Former owner,Engraver,Issuance type,Flickr URL,Shelfmarks
0,206,NaN,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12641.b.30.
1,216,NaN,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12626.cc.2.
2,218,NaN,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12625.dd.1.
3,472,NaN,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.","Appleyard, Ernest Silvanus.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 10369.bbb.15.
4,480,"A new edition, revised, etc.",London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.","BROOME, John Henry.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 9007.d.28.


## Dropping Columns with .drop()


You can see that a lot of this columns don't really provide the information on the books we are looking for. Let's drop the following columns `Edition Statement, Corporate Author, Corporate Contributors, Former owner, Engraver, Issuance type, Shelfmarks`

Often, we encounter columns that aren't useful for our analysis. To remove them, we can use the `.drop()` method by passing a list of the column names we want to delete.



To clean up the DataFrame and remove the columns we don't need, we'll first store the column names in a list (to_drop). Then, we'll use the `.drop()` method to remove them from the DataFrame. Adding the`axis=1` to the drop command tells pandas to drop columns not rows (`axis=0`), and
`inplace=True` applies the changes directly to the original DataFrame.



In [9]:
to_drop = ['Edition Statement',
            'Corporate Author',
            'Corporate Contributors',
            'Former owner',
            'Engraver',
            'Contributors',
            'Issuance type',
            'Shelfmarks']

df.drop(to_drop, inplace=True, axis=1)

When we inspect our dataset we see that the unwanted columns have been removed.

In [10]:
df.head()

,Identifier,Place of Publication,Date of Publication,Publisher,Title,Author,Flickr URL
0,206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,http://www.flickr.com/photos/britishlibrary/ta...
1,216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
2,218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
3,472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...
4,480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...


We could also use an alternative syntax to drop our columns.

In [11]:
#df.drop(columns=to_drop, inplace=True) # will throwback error because we already ran this so it has no columns to drop and is an alternative way to do this

## Changing the index
Remember we can index our dataframes with any of the columns in the DF, this is versatile, but does not always guarantee unique indexes, which would be a problem if you have two books with the same name.

First let's check if our identifier column is unique with the `.is_unique` method. This will return a True or False on a specified column.

In [12]:
df['Identifier'].is_unique

True

Let's set our index to the `Identifier` column.

In [13]:
df = df.set_index('Identifier')
df.head()

,Place of Publication,Date of Publication,Publisher,Title,Author,Flickr URL
Identifier,,,,,,
206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,http://www.flickr.com/photos/britishlibrary/ta...
216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...
480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...


We can access each record using `loc[]` and an index number.

In [14]:
df.loc[480]

,480
Place of Publication,London
Date of Publication,1857
Publisher,Wertheim & Macintosh
Title,"[The World in which I live, and my place in it..."
Author,"A., E. S."
Flickr URL,http://www.flickr.com/photos/britishlibrary/ta...


Previously, our index was a RangeIndex: integers starting from 0, analogous to Python’s built-in range. By passing a column name to set_index, we have changed the index to the values in Identifier.

We could also use the `set_index()` method with the `inplace=True` parameter so our changes affect the dataframe directly and we don't have to create a new copy.

In [15]:
# Running this will result in an error...
# ...if index is already set to Identifier
#df.set_index('Identifier', inplace=True)

## Cleaning Datafields
In this section we will format our data so it has a more consisten format. We will be changing the `Date of Publication` and `Place of Publication`.

If we take a look at our `Date of Publication` we can see that we have `[]` square brackets, multiple publication dates. We will need to:
- Remove the extra dates in the square brackets `[1878]`.
- Convert dates to `start date` whenever present.
- Remove dates we are not certain about

In [16]:
 df.loc[1905:, 'Date of Publication'].head(10)

,Date of Publication
Identifier,
1905,1888
1929,"1839, 38-54"
2836,1897
2854,1865
2956,1860-63
2957,1873
3017,1866
3131,1899
4598,1814


Synthesizing these patterns, we can actually take advantage of a single regular expression to extract the publication year:

The regular expression above is meant to find any four digits at the beginning of a string, which suffices for our case. The above is a raw string (meaning that a backslash is no longer an escape character), which is standard practice with regular expressions.

The \d represents any digit, and {4} repeats this rule four times. The ^ character matches the start of a string, and the parentheses denote a capturing group, which signals to Pandas that we want to extract that part of the regex. (We want ^ to avoid cases where `[` starts off the string.)

In [17]:
# Running our regex with the .str method
extr = df['Date of Publication'].str.extract(r'^(\d{4})', expand=False)
extr.head(20)

,Date of Publication
Identifier,
206,1879
216,1868
218,1869
472,1851
480,1857
481,1875
519,1872
667,NaN
874,1676


This column still has object dtype, but we can easily get its numerical version with `pd.to_numeric`

In [18]:
df['Date of Publication'] = pd.to_numeric(extr)
df['Date of Publication'].dtype

dtype('float64')

This results in about one in every ten values being missing, which is a small price to pay for now being able to do computations on the remaining valid values:

In [19]:
df['Date of Publication'].isnull().sum() / len(df)

np.float64(0.11717147339205986)

### `str` Methods with Numpy to Clean Columns

you may have noticed the use of `df['Date of Publication']``.str.` This attribute is a way to access speedy string operations in Pandas that largely mimic operations on native Python strings or compiled regular expressions, such as `.split()`, `.replace()`, and `.capitalize()`.

To clean the Place of Publication field, we can combine Pandas str methods with NumPy’s np.where function, which is basically a vectorized form of Excel’s IF() macro. It has the following syntax:

`np.where(condition, then, else)`

Essentially, `.where()` takes each element in the object used for condition, checks whether that particular element evaluates to True in the context of the condition, and returns an ndarray containing then or else, depending on which applies.

It can be nested into a compound if-then statement, allowing us to compute values based on multiple conditions:

` np.where(condition1, x1,
        np.where(condition2, x2,
            np.where(condition3, x3, ...)))`

In [20]:
# Check our place of publication column
df['Place of Publication'].head(10)

,Place of Publication
Identifier,
206,London
216,London; Virtue & Yorston
218,London
472,London
480,London
481,London
519,London
667,"pp. 40. G. Bryan & Co: Oxford, 1898"
874,London]


We see that for some rows, the place of publication is surrounded by other unnecessary information. If we were to look at more values, we would see that this is the case for only some rows that have their place of publication as ‘London’ or ‘Oxford’.

In [21]:
df.loc[4157862]

,4157862
Place of Publication,Newcastle-upon-Tyne
Date of Publication,1867.0
Publisher,T. Fordyce
Title,"Local Records; or, Historical Register of rema..."
Author,"FORDYCE, T. - Printer, of Newcastle-upon-Tyne"
Flickr URL,http://www.flickr.com/photos/britishlibrary/ta...


In [22]:
df.loc[4159587]

,4159587
Place of Publication,Newcastle upon Tyne
Date of Publication,1834.0
Publisher,Mackenzie & Dent
Title,"An historical, topographical and descriptive v..."
Author,"Mackenzie, E. (Eneas)"
Flickr URL,http://www.flickr.com/photos/britishlibrary/ta...


These two books were published in the same place, but one has hyphens in the name of the place while the other does not.

To clean this column in one sweep, we can use str.contains() to get a Boolean mask.

We clean the column as follows:

In [23]:
pub = df['Place of Publication']
london = pub.str.contains('London')
london[:5]

,Place of Publication
Identifier,
206,True
216,True
218,True
472,True
480,True


In [24]:
# Combine it with np.where
oxford = pub.str.contains('Oxford')

In [25]:
df['Place of Publication'] = np.where(london, 'London',
                                      np.where(oxford, 'Oxford',
                                                                 pub.str.replace('-', ' ')))

In [26]:
df['Place of Publication'].head()

,Place of Publication
Identifier,
206,London
216,London
218,London
472,London
480,London


Here, the np.where function is called in a nested structure, with condition being a Series of Booleans obtained with str.contains(). The contains() method works similarly to the built-in in keyword used to find the occurrence of an entity in an iterable (or substring in a string).

The replacement to be used is a string representing our desired place of publication. We also replace hyphens with a space with str.replace() and reassign to the column in our DataFrame.

Although there is more dirty data in this dataset, we will discuss only these two columns for now.

Let’s have a look at the first five entries, which look a lot crisper than when we started out:

In [27]:
df.head()

,Place of Publication,Date of Publication,Publisher,Title,Author,Flickr URL
Identifier,,,,,,
206,London,1879.0,S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,http://www.flickr.com/photos/britishlibrary/ta...
216,London,1868.0,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
218,London,1869.0,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.",http://www.flickr.com/photos/britishlibrary/ta...
472,London,1851.0,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...
480,London,1857.0,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.",http://www.flickr.com/photos/britishlibrary/ta...


### Using the `applymap` Function
In certain situations, you will see that the dirty data is not localized to one column but is more spread out.

There are some instances where it would be helpful to apply a customized function to each cell or element of a DataFrame. Pandas `.applymap()` method is similar to the in-built `map()` function and simply applies a function to all the elements in a DataFrame.

Let’s look at an example. We will create a DataFrame out of the `“university_towns.txt”` file:

we have periodic state names followed by the university towns in that state: StateA TownA1 TownA2 StateB TownB1 TownB2.... If we look at the way state names are written in the file, we’ll see that all of them have the “[edit]” substring in them.

We can take advantage of this pattern by creating a list of (state, city) tuples and wrapping that list in a DataFrame:

In [28]:
university_towns = []
>>> with open('university_towns.txt') as file:
...     for line in file:
...         if '[edit]' in line:
...             # Remember this `state` until the next is found
...             state = line
...         else:
...             # Otherwise, we have a city; keep `state` as last-seen
...             university_towns.append((state, line))

In [29]:
university_towns[:5]

[('Alabama[edit]\n', 'Auburn (Auburn University)[1]\n'),
 ('Alabama[edit]\n', 'Florence (University of North Alabama)\n'),
 ('Alabama[edit]\n', 'Jacksonville (Jacksonville State University)[2]\n'),
 ('Alabama[edit]\n', 'Livingston (University of West Alabama)[2]\n'),
 ('Alabama[edit]\n', 'Montevallo (University of Montevallo)[2]\n')]

We can wrap this list in a DataFrame and set the columns as “State” and “RegionName”. Pandas will take each element in the list and set State to the left value and RegionName to the right value.

The resulting DataFrame looks like this:

In [30]:
towns_df = pd.DataFrame(university_towns,
                         columns=['State', 'RegionName'])

In [31]:
towns_df.head()

,State,RegionName
0,Alabama[edit]\n,Auburn (Auburn University)[1]\n
1,Alabama[edit]\n,Florence (University of North Alabama)\n
2,Alabama[edit]\n,Jacksonville (Jacksonville State University)[2]\n
3,Alabama[edit]\n,Livingston (University of West Alabama)[2]\n
4,Alabama[edit]\n,Montevallo (University of Montevallo)[2]\n


While we could have cleaned these strings in the for loop above, Pandas makes it easy. We only need the state name and the town name and can remove everything else. While we could use Pandas’ .str() methods again here, we could also use applymap() to map a Python callable to each element of the DataFrame.

We have been using the term element, but what exactly do we mean by it? Consider the following “toy” DataFrame:

|        0|           1|
|---|---|
|0|    Mock|     Dataset|
|1  |Python|     Pandas|
|2   | Real |    Python|
|3   |NumPy  |   Clean|

In this example, each cell (‘Mock’, ‘Dataset’, ‘Python’, ‘Pandas’, etc.) is an element. Therefore, applymap() will apply a function to each of these independently. Let’s define that function:

In [32]:
def get_citystate(item):
     if ' (' in item:
        return item[:item.find(' (')]
     elif '[' in item:
        return item[:item.find('[')]
     else:
        return item

Pandas’ .applymap() only takes one parameter, which is the function (callable) that should be applied to each element:

In [33]:
towns_df =  towns_df.applymap(get_citystate)

/tmp/ipython-input-33-1572465724.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  towns_df =  towns_df.applymap(get_citystate)


First, we define a Python function that takes an element from the DataFrame as its parameter. Inside the function, checks are performed to determine whether there’s a ( or [ in the element or not.

Depending on the check, values are returned accordingly by the function. Finally, the applymap() function is called on our object. Now the DataFrame is much neater:



In [34]:
towns_df.head(20)

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


The applymap() method took each element from the DataFrame, passed it to the function, and the original value was replaced by the returned value.

### Renaming Columns and Skipping Rows
Often, the datasets you’ll work with will have either column names that are not easy to understand, or unimportant information in the first few and/or last rows, such as definitions of the terms in the dataset, or footnotes.

In that case, we’d want to rename columns and skip certain rows so that we can drill down to necessary information with correct and sensible labels.

To demonstrate how we can go about doing this, let’s first take a glance at the initial five rows of the “olympics.csv” dataset:

## Lesson 4b: Merge and Joins
Pandas allows various methods to merge and join dataframes. This allows you to create efficient Dataframes from various sources
- `pd.merge()`
- `pd.join()`
- `pd.concat()`

We will use a display function to correctly display our Dataframes

In [35]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)

    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [36]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


## Categories of Joins

The `pd.merge` function implements a number of types of joins:
- `one-to-one`
- `many-to-one`
- `many-to-many`

 All three types of joins are accessed via an identical call to the `pd.merge`interface; the type of join performed depends on the form of the input data.

We'll start with some simple examples of the three types of merges, and discuss detailed options a bit later.

### One-to-one Joins

One of the most straightforward types of merges is the one-to-one join. It closely resembles a column-wise combination, like what you saw with concat() in previous lessons when combining datasets.

Lets combine our dataframes with `pd.merge()`. Our dataframe is automatically joined by the employee column.

df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

In [37]:
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


The `pd.merge()` function automatically detects that both DataFrames have a column named employee and uses it as the key to combine them. The output is a new DataFrame that brings together data from both sources.

Notice that the original row order from each DataFrame may not be preserved — for instance, if the employee column appears in a different order in df1 and df2, pd.merge() still aligns the rows correctly based on matching values.

Additionally, keep in mind that the merge in general discards the index, except in the special case of merges by index which we will discuss later in the lesson.

### Many-to-one Joins

Many-to-one joins occur when one of the key columns has repeated values. In these cases, the resulting DataFrame retains those duplicates, matching each one appropriately with the corresponding entry from the other DataFrame.

Here one of the two columns contain duplicate entries

In [38]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

pd.merge(df3, df4)
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

The resulting DataFrame includes a new column containing the "supervisor" information, which is repeated wherever necessary to match the corresponding entries from the original data.

### Many-to many Joins


Many-to-many joins can be a bit tricky to understand at first, but they follow a clear logic. When both DataFrames have duplicate values in the key column, the merge operation performs a many-to-many match. This means that each matching pair is included in the result.

A clear example helps make this concept more intuitive: imagine one DataFrame lists multiple skills for each group, and another links individuals to groups. A many-to-many join can then be used to retrieve all the skills associated with each individual based on their group membership.

Here both of the two columns contain duplicate entries

In [39]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


Simple merge on employee column. This only works if the `employee` column exists

 ## Specify the Merge Keys

By default, `pd.merge` searches for columns with matching names in both DataFrames and uses those as the merge key. However, in many cases, the column names won’t line up perfectly — and fortunately, `pd.merge` offers several options to handle these situations.

### The `on` Keywords
The simplest way to define the merge key is by using the `on` keyword. This allows you to directly specify the column (or list of columns) that should be used to match rows between the two DataFrames.


In [40]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

**Note: This option works only if both the left and right DataFrames have the specified column name.**

If column names are different on both dataframes, we can use the `left_on` to specify the left column, and `right_on` to specify the other column.

In [41]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1, df3, left_on="employee", right_on="name")
  employee        group  name  salary
0      Bob   Accounting   Bob   70000
1     Jake  Engineering  Jake   80000
2     Lisa  Engineering  Lisa  120000
3      Sue           HR   Sue   90000

The result has a redundant column that we can drop if desired–for example, by using the `drop()` method

In [42]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


 The `left_index` and `right_index` Keywords

Sometimes, rather than merging on a column, you would instead like to merge on an index. For example, your data might look like this:

In [43]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


You can use the index as the key for merging by specifying the `left_index` and/or `right_index` flags in `pd.merge()`

In [44]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


For convenience, DataFrames implement the `join()` method, which performs a merge that defaults to joining on indices:

In [45]:
display('df1a', 'df2a', 'df1a.join(df2a)')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


If you'd like to mix indices and columns, you can combine `left_index` with `right_on` or `left_on` with `right_index` to get the desired behavior

In [46]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

pd.merge(df1a, df3, left_index=True, right_on='name')
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000

## Specifying Set Arithmetic for Joins

In all the preceding examples we have glossed over one important consideration in performing a join: the type of set arithmetic used in the join. This comes up when a value appears in one key column but not the other.

In [47]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7)
   name   food drink
0  Mary  bread  wine

Here we have merged two datasets that have only a single "name" entry in common: Mary. By default, the result contains the intersection of the two sets of inputs; this is what is known as an inner join. We can specify this explicitly using the `how` keyword, which defaults to `"inner"`.

In [48]:
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


Other options for the how keyword are `'outer', 'left', and 'right'`. An outer join returns a join over the union of the input columns, and fills in all missing values with NAs:

In [49]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='outer')
     name   food drink
0  Joseph    NaN  beer
1    Mary  bread  wine
2    Paul  beans   NaN
3   Peter   fish   NaN

The left join and right join return joins over the left entries and right entries, respectively. For example:

In [50]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

The output rows now correspond to the entries in the left input. Using how='right' works in a similar manner.

All of these options can be applied straightforwardly to any of the preceding join types.

In [51]:
olympics_df = pd.read_csv('olympics.csv')
olympics_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,? Summer,01 !,02 !,03 !,Total,? Winter,01 !,02 !,03 !,Total,? Games,01 !,02 !,03 !,Combined total
1,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
2,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
3,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
4,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12


This is messy indeed! The columns are the string form of integers indexed at 0. The row which should have been our header (i.e. the one to be used to set the column names) is at olympics_df.iloc[0]. This happened because our CSV file starts with 0, 1, 2, …, 15.

Also, if we were to go to the [source](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table) of this dataset, we’d see that NaN above should really be something like “Country”, ? Summer is supposed to represent “Summer Games”, 01 ! should be “Gold”, and so on.

Therefore, we need to do two things:

- Skip one row and set the header as the first (0-indexed) row
- Rename the columns

We can skip rows and set the header while reading the CSV file by passing some parameters to the read_csv() function.

This function takes a lot of optional parameters, but in this case we only need one (header) to remove the 0th row:

In [52]:
olympics_df = pd.read_csv('olympics.csv', header=1)
olympics_df.head()

,Unnamed: 0,? Summer,01 !,02 !,03 !,Total,? Winter,01 !.1,02 !.1,03 !.1,Total.1,? Games,01 !.2,02 !.2,03 !.2,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


We now have the correct row set as the header and all unnecessary rows removed. Take note of how Pandas has changed the name of the column containing the name of the countries from NaN to Unnamed: 0.

To rename the columns, we will make use of a DataFrame’s rename() method, which allows you to relabel an axis based on a mapping (in this case, a dict).

Let’s start by defining a dictionary that maps current column names (as keys) to more usable ones (the dictionary’s values):

In [53]:
new_names =  {'Unnamed: 0': 'Country',
               '? Summer': 'Summer Olympics',
               '01 !': 'Gold',
               '02 !': 'Silver',
               '03 !': 'Bronze',
               '? Winter': 'Winter Olympics',
               '01 !.1': 'Gold.1',
               '02 !.1': 'Silver.1',
               '03 !.1': 'Bronze.1',
               '? Games': '# Games',
               '01 !.2': 'Gold.2',
               '02 !.2': 'Silver.2',
               '03 !.2': 'Bronze.2'}

In [54]:
# Call the rename function
olympics_df.rename(columns=new_names, inplace=True)
olympics_df.head()

,Country,Summer Olympics,Gold,Silver,Bronze,Total,Winter Olympics,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


## 🛑 End of Lesson 4a and 4b covering Data preporcessing , Merges and Joins

## **Please proceed to Lesson 4c: Data Visualization to continue the tutorial.**

Please complete the following Tasks:

- ✅ **Lesson 4 Terminology Quiz**
- ✅ **Lesson 4 Coding Exercises**

📩 Submit all solutions to your instructor once complete.